In [ ]:
import pandas as pd
import numpy as np
import warnings
import re
import math
warnings.filterwarnings("ignore")

In [ ]:
# подгрузка всех файлов испытаний
import os
dr = 'exp'
file_names = os.listdir(dr)

In [ ]:
del file_names[-3]

In [ ]:
# файлы в словарь доступ по имени файла ({имя файла: словарь датафреймов файла} )
exp_files = {}
for name in file_names:
    exp_files[name] = pd.read_excel(dr+'\\'+name, None)
    print(name)

In [ ]:
tmp = {}
for name in exp_files:
    for l in exp_files[name]:
        if '№ плавки' in exp_files[name][l]:
            exp_files[name][l]['№ плавки'] = exp_files[name][l]['№ плавки'].astype(str).str.lower()
        if '№ партии' in exp_files[name][l]:
            exp_files[name][l]['№ партии'] = exp_files[name][l]['№ партии'].astype(str).str.lower()
    tmp[name] = exp_files[name]
exp_files = tmp

In [ ]:
ls_mean = [
        'кремний (8)', 'углерод (4)', 'азот (5)', 'сера (10)', 'ванадий (13)',
        'хром (14)', 'марганец (15)', 'алюминий (7)', 'никель (18)',
        'медь (19)', 'молибден (25)', 'титан (12)', 'фосфор (9)', 'ниобий (24)',
        'бор (3)', 'водород (36)', 'кислород (37)' 
]

In [ ]:
# создаем единую таблицу с химией
chemistry = []
chemistry = exp_files['1квартал2017.xls'][u'Хим.анализ плавок (заготовка)']
# print(chemistry)
chemistry['имя файла'] = '1квартал2017.xls'
chemistry_list_names = [
    'Хим.анализ плавок (заготовка)',
    'Хим.анализ плавок(заготовка)',
    'Хим.анализ плавок (Заготовка)',
    'Хим.анализ плавок (заготовки)' 
]


s = 0
for name, exp_file in zip(exp_files, exp_files.values()):
    for list_chem in chemistry_list_names:
        if list_chem in exp_file:
            tmp = exp_file[list_chem].copy()
            tmp['имя файла'] = name
            chemistry = pd.merge(chemistry, tmp, how = 'outer')
            s += exp_file[list_chem].shape[0]
            break

chemistry[u'№ плавки'] = chemistry[u'№ плавки'].astype(str)
chemistry.to_excel('chemistry.xlsx')

print(s)
print(chemistry.shape)

In [ ]:
chem_list_without_filename = list(chemistry.columns)
chem_list_without_filename.remove('имя файла')
chemistry.drop_duplicates(chem_list_without_filename, inplace=True)
chemistry.shape

In [ ]:
chemistry = chemistry[chemistry['Поставщик']=='ЭСПК']
# chemistry.columns


In [ ]:
chemistry[chemistry[['№ плавки']].duplicated(keep = False)].sort_values(['№ плавки'])

In [ ]:
def ch2f(x):
    if (type(x)==float) | (type(x)==int):
        x = str(x)
    if u'г' in x:
        print(x)
        x = x.replace(u'г.п.', '')
        x = x.replace(u'гп', '')
        x = str(x)
    if u'0.012. 0.014. 0.013' in str(x):
        x = x.replace(u'0.012. 0.014. 0.013', u'0.013')
    if x[-4:-2]=='e-':
        return float(x)
    elif '-' in str(x): 
#         print(x)
        x = x.replace(' ', '')
        x = str(x) + '0'
        tmp_map = list(map(float, str(x).replace(',', '.').split('-')))
        if (abs(tmp_map[-1]-tmp_map[-2])>0.05) | (tmp_map[-1]==0):
            return np.mean(tmp_map[:-1])
        else:
            return np.mean(tmp_map)
    else:
        return float(str(x).replace(',', '.'))

    
chem_list = [
    u'C',
    u'Mn',
    u'Si',
    u'P',
    u'S',
    u'Cr',
    u'Ni',
    u'Cu',
    u'Al',
    u'V',
    u'Ti',
    u'Nb',
    u'Mo',
    u'N',
    u'B'
]

In [ ]:
err_chem_df = exp_files['1квартал2017.xls'][u'Хим.анализ плавок (заготовка)'][
    exp_files['1квартал2017.xls'][u'Хим.анализ плавок (заготовка)']['Al'].apply(ch2f)>0.06]
err_chem_df['имя файла'] ='1квартал2017.xls'
# err_chem_df['problem'] = 'Al'


def add_borders(err_chem_df, el, border):
    count = 0 
    for name, exp_file in zip(exp_files, exp_files.values()):
        for chem in chemistry_list_names:
            if chem in exp_file:
                if el in exp_file[chem]:
                    tmp = exp_file[chem][exp_file[chem][el].apply(ch2f)<border]
                    if tmp.shape[0]>0:
#                         tmp['problem'] = el
                        tmp['имя файла'] = name
                        err_chem_df = pd.merge(err_chem_df, tmp, how='outer')
                        count += tmp.shape[0]
    print(count)
    return err_chem_df


err_chem_df = add_borders(err_chem_df, 'Al', 0.06)
err_chem_df = add_borders(err_chem_df, 'V', 0.13) 
err_chem_df = add_borders(err_chem_df, 'Nb', 0.09)
err_chem_df = add_borders(err_chem_df, 'Mo', 0.25)
err_chem_df = add_borders(err_chem_df, 'N', 0.015)
err_chem_df = add_borders(err_chem_df, 'B', 0.001)


def save_df_width_col(df, file_name, width=80):
    """Сохраняет датафрейм с широкой первой ячейкой """
    writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
    df.to_excel(writer)
    sh = writer.sheets['Sheet1']
    sh.set_column(1, 1,width)
    writer.close()

    
columns_list = list(err_chem_df.columns)  
columns_list[0], columns_list[-2] = columns_list[-2], columns_list[0]
columns_list[-3], columns_list[-2], columns_list[-1]  = columns_list[-2], columns_list[-1], columns_list[-3]
err_chem_df = err_chem_df[columns_list]

In [ ]:
chemistry.drop_duplicates(chem_list_without_filename, inplace=True)

chemistry = chemistry[err_chem_df['Поставщик']=='ЭСПК']
chemistry.shape

In [ ]:
list_err_numbers = list(err_chem_df['№ плавки'].value_counts().index)


In [ ]:
def add_steel_mark(df):
    for i in range(0, df.shape[0]):
        fl = False
        exp_file = exp_files[df['имя файла'][i]]
        for list_name in exp_file.keys():
            if 'Марка стали' in exp_file[list_name]:
                for j in range(0, exp_file[list_name].shape[0]):
                    if exp_file[list_name]['№ плавки'][j]==df['№ плавки'][i]:
                        df.loc[i, 'Марка стали'] = exp_file[list_name]['Марка стали'][j]
                        fl = True
                    if fl:
                        break
            if fl:
                break
    return df

In [ ]:
err_chem_df = add_steel_mark(err_chem_df)    
# err_chem_df.reset_index(drop=True, inplace=True)

In [ ]:
err_chem_df.to_excel('chem_mark_file.xlsx')

In [ ]:
save_df_width_col(err_chem_df.sort_values('№ плавки'), 'chem_errors_sort_by_number.xlsx')
save_df_width_col(err_chem_df.sort_values('имя файла'), 'chem_errors_sort_by_file_name.xlsx')

In [ ]:
for element in chem_list:
    chemistry[element] = chemistry[element].apply(ch2f)
chemistry.shape

In [ ]:
print(chemistry.shape)
chemistry = chemistry.drop_duplicates()
print(chemistry.shape)

In [ ]:
# chemistry = get_mean(chemistry)
chemistry.shape

In [ ]:
# Партии-дубликаты с близкими значениями по углероду
dupl_df = chemistry[chemistry['№ плавки'].duplicated(keep = False)].sort_values(['№ плавки'])
def c_dif(x):
    fl = False
    for i in x['C']:
        for j in x['C']:
            if (abs(i-j)<0.1) & (abs(i-j)>0):
                fl = True
    x['C_dif<0.1']= fl
#     print(x)
    return x
dupl_df.reset_index(drop=True, inplace=True)    
dupl_df['C_dif<0.1']=None
dupl_df = dupl_df.groupby('№ плавки').apply(c_dif)
dupl_df.reset_index(drop=True, inplace=True)
dupl_df[dupl_df['C_dif<0.1']==True].shape
# dupl_df[dupl_df['C_dif<0.1']==True].to_excel('check_dubl_c_dif_01.xlsx')

In [ ]:
mode = pd.read_excel('all_modes_clean_2.xlsx')
mode.shape
mode[u'№ плавки'] = mode[u'№ плавки'].astype(str)
mode.shape

In [ ]:
# mode.shape
# mode[mode[['№ плавки', '№ партии']].duplicated(keep = False)].sort_values('№ партии')

In [ ]:
merge_table = pd.merge(mode, chemistry, how ='left', on=['№ плавки'])
print(merge_table.shape)

In [ ]:
# merge_table.columns

In [ ]:
for name, f in zip(exp_files, exp_files.values()):
    for l in chemistry_list_names:
        if l in f:
            if '№ плавки' in f[l]:
                for i in f[l]['№ плавки']:
                    if i  == 'v20694':
                        print(l, name)

In [ ]:
merge_table.to_excel('merge_table_new.xlsx')

In [ ]:
mean_chem_steel = pd.read_excel('mean_chem_steel.xlsx')

In [ ]:
merge_table = pd.read_excel('merge_table_new.xlsx')

In [ ]:
merge_table.shape, mean_chem_steel.shape

In [ ]:
merge_table[merge_table['C'].isnull()].shape

In [ ]:
ls = merge_table.columns[:-15]
merge_without_chem = merge_table[merge_table['C'].isnull()]
print(merge_without_chem.shape)
merge_without_chem = merge_without_chem[ls]
merge_without_chem.shape

In [ ]:
merge_table = merge_table[merge_table['C'].notnull()]
merge_table.shape

In [ ]:
merge_mean_chem = pd.merge(merge_without_chem, mean_chem_steel, how='left', on='марка стали')

In [ ]:
merge_mean_chem.shape
# merge_mean_chem

In [ ]:
merge_table = pd.merge(merge_table, merge_mean_chem, how ='outer')
merge_table.shape

In [ ]:
"""
Усредняем близкие по химии плавки, и оставляем одну из двух 
(пока первую, потом по марке можно будет прикрутить) для далеких (если разница между любыми хим элементами больше 0.01)
"""
def get_need_chem_list(mark_list):
    c_need_list = []
    for i in mark_list:
        if (i=='Д')|(i=='ДБ'):
            c_need_list.append(float('0.45'))
        elif (type(i)!=float) & (i!='ГОСТ Р53366 "N80 Q"') & (i!='ТУ 1319-369'):
            c_need = float(str(i)[0:2])/100
            c_need_list.append(c_need)
        else:
            c_need_list.append(float('nan'))
    return c_need_list


def to_mean_or_not_to_mean(x):
    c_need_list = get_need_chem_list(x['Марка стали'])
    
    is_different = False
    for ch in chem_list:
        list_ch = []
        for i in x[ch]:
            list_ch.append(i)
        for i in x[ch]:
            for j in x[ch]:
                if (abs(i-j)>0.01)|(is_different):
                    is_different = True
    
    if is_different:
        list_C = []
        for c in x['C']:
            list_C.append(c)
        index = -1
        min_dif = 0.04
        for i in range(0,len(list_C)):
            for c_n in c_need_list:
                if abs(c_n-list_C[i])<min_dif:
                    min_dif=abs(c_n-list_C[i])
                    index = i
        for ch in chem_list:
            list_ch = []
            for i in x[ch]:
                list_ch.append(i)
            for i in x[ch]:
                for j in x[ch]:
                    if (abs(i-j)>0.01)|(is_different):
                        is_different = True
                        x[ch] = list_ch[index]
            x[ch] = x[ch].mean()
    else:
        x[ch] = x[ch].mean()
    return x


def get_mean(df):
    df = df.groupby('№ плавки').apply(to_mean_or_not_to_mean)
    df.reset_index(drop=True, inplace=True)
    return df

In [ ]:
merge_table['C'] .describe()

In [ ]:
def switch_c_mn(x):
    if x['C']>=0.9:
        tmp = x['C']
        x['C']= x['Mn']
        x['Mn'] = tmp
        print(x['C'], x['Mn'])
    return x
merge_table = merge_table.apply(switch_c_mn, axis =1)

In [ ]:
merge_table['C'].describe()

In [ ]:
print(merge_table.shape)
merge_table = get_mean(merge_table)
print(merge_table.shape)

In [ ]:
merge_table.drop_duplicates(chem_list+['№ плавки', '№ партии'], inplace=True)
merge_table.shape

In [ ]:
err_chem_df[err_chem_df[['№ плавки']].duplicated(keep = False)].sort_values(['№ плавки'])
# .to_excel('check_dupl.xlsx')

In [ ]:
merge_table.shape

In [ ]:
merge_table.drop_duplicates(['№ плавки', '№ партии'], inplace=True)
merge_table.shape

In [ ]:
# merge_table = pd.read_excel('merge_table_mean_chem.xlsx')

In [ ]:
copy = merge_table[['марка стали']+chem_list+['№ плавки', '№ партии']]

In [ ]:
merge_table.shape

In [ ]:
merge_table = pd.concat([merge_table[merge_table['Al']<0.08], merge_table[merge_table['Al'].isnull()]])
merge_table.reset_index(drop=True, inplace=True)

In [ ]:
copy[copy['№ плавки']=='434396']

In [ ]:
copy = copy[copy['B']>0.0005]
print(copy.shape)
copy['марка стали'].unique()
# copy

In [ ]:
merge_table.shape

In [ ]:
merge_table.to_excel('merge_table_mean_chem.xlsx')

In [ ]:
copy.to_excel('check_chem.xlsx')